### Here I'm trying to check necessary number of integrals

In [1]:
import numpy as np
from source_helper import *
import matplotlib.pyplot as plt
% matplotlib inline
from time import time
from lapl_well import LaplWell
from helper import Helper
from well import Well
import pandas as pd

In [2]:
# define a well in real space
xwd = 0.
ywd = 0.
outer_bound = "infinite"
top_bound = "imp"
bottom_bound = "imp"
wtype = "frac"
n_stehf = 12
params = {"nseg": 20, "kx": 1., "ky": 1., "kz": 1., "omega": 1, "lambda": 1, "Fcd":10, "ref_length":1.,
         "z_ref_length": 1}
real_well = Well(xwd, ywd, outer_bound, top_bound, bottom_bound, wtype, params, n_stehf)
well = real_well.lapl_well

In [3]:
helper = Helper()

In [25]:
h_lims, h_ulims = helper.get_limits(well)

In [30]:
low_alims = well.geokeeper.matrices["low_alims"]
high_alims = well.geokeeper.matrices["high_alims"]
u_ulims = well.geokeeper.matrices["ulims"]

In [31]:
u_ulims

array([ 0.025,  0.075,  0.125,  0.175,  0.225,  0.275,  0.325,  0.375,
        0.425,  0.475,  0.525,  0.575,  0.625,  0.675,  0.725,  0.775,
        0.825,  0.875,  0.925,  0.975,  1.025,  1.075,  1.125,  1.175,
        1.225,  1.275,  1.325,  1.375,  1.425,  1.475,  1.525,  1.575,
        1.625,  1.675,  1.725,  1.775,  1.825,  1.875,  1.925,  1.975])

In [32]:
h_ulims

array([ 0.025,  0.075,  0.125,  0.175,  0.225,  0.275,  0.325,  0.375,
        0.425,  0.475,  0.525,  0.575,  0.625,  0.675,  0.725,  0.775,
        0.825,  0.875,  0.925,  0.975,  1.025,  1.075,  1.125,  1.175,
        1.225,  1.275,  1.325,  1.375,  1.425,  1.475,  1.525,  1.575,
        1.625,  1.675,  1.725,  1.775,  1.825,  1.875,  1.925,  1.975])

In [33]:
u_ulims == h_ulims

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True], dtype=bool)

In [198]:
def get_limits(well, s):
    N = well.params["nseg"] # number of elements in half-length
    m = -1*np.ones((2*N, 2*N, 2))
    dx = 1./N
    su = s**0.5
    c = 1
    xwd = well.xwd
    if well.wtype in ["frac", "vertical"]:
        zd = 0
    for i in range(-N, N):
        xd = well.xwd + dx*(i + 0.5)
        for j in range(-N, N):
            int_lim1 = well.xwd + j*dx
            int_lim2 = well.xwd + (j + 1)*dx
            if (xd - xwd - c*int_lim2) >=0:
                m[i+N,j+N,0] = su*(xd-xwd-c*int_lim1)
                m[i+N,j+N,1] = su*(xd-xwd-c*int_lim2)
            elif (xd - xwd - c*int_lim1) <0:
                m[i+N,j+N,0] = su*(c*int_lim2+xwd-xd)
                m[i+N,j+N,1] = su*(c*int_lim1+xwd-xd)
            else:
                m[i+N,j+N,0] = su*(xd-xwd-c*int_lim1)
                m[i+N,j+N,1] = su*(c*int_lim2+xwd-xd)
    return np.around(m, decimals=6)

In [199]:
lims = get_limits(well, 1)
ulims = np.unique(lims0)
ints = iti0k0(ulims)[1]
lims0 = lims[...,0]
lims0.shape
ulims.shape

(40,)

In [200]:
base_lims = get_limits(well, 1)[...,0]
base_ulims = np.unique(base_lims)
N = well.params["nseg"]
for i in range(1):
    ss = 1000*np.random.sample(10000)
    ss = ss[ss != 0.]
    t1 = time()
    for s in ss:
        lims = (s**0.5)*base_lims
        x = iti0k0(lims)[1]
    slow_time = time() - t1
    t1 = time()
    for s in ss:
        ulims = (s**0.5)*base_ulims
        ints = iti0k0(ulims)[1]
        x = ints[np.argwhere(base_ulims == base_lims.reshape(4*N*N,1))[:,1]].reshape(2*N,2*N)
    fast_time = time()-t1
    print(slow_time, fast_time)

114.5485520362854 175.59004306793213


In [201]:
l

NameError: name 'l' is not defined

In [183]:
ints[np.argwhere(ulims == lims0.reshape(1600,1))[:,1]].reshape(40,40)

array([[ 0.12012696,  0.27810695,  0.39999634, ...,  1.45828363,
         1.46472704,  1.4707842 ],
       [ 0.27810695,  0.12012696,  0.27810695, ...,  1.45142734,
         1.45828363,  1.46472704],
       [ 0.39999634,  0.27810695,  0.12012696, ...,  1.44412948,
         1.45142734,  1.45828363],
       ..., 
       [ 1.45828363,  1.45142734,  1.44412948, ...,  0.12012696,
         0.27810695,  0.39999634],
       [ 1.46472704,  1.45828363,  1.45142734, ...,  0.27810695,
         0.12012696,  0.27810695],
       [ 1.4707842 ,  1.46472704,  1.45828363, ...,  0.39999634,
         0.27810695,  0.12012696]])

In [179]:
iti0k0(lims0)[1]

array([[ 0.12012696,  0.27810695,  0.39999634, ...,  1.45828363,
         1.46472704,  1.4707842 ],
       [ 0.27810695,  0.12012696,  0.27810695, ...,  1.45142734,
         1.45828363,  1.46472704],
       [ 0.39999634,  0.27810695,  0.12012696, ...,  1.44412948,
         1.45142734,  1.45828363],
       ..., 
       [ 1.45828363,  1.45142734,  1.44412948, ...,  0.12012696,
         0.27810695,  0.39999634],
       [ 1.46472704,  1.45828363,  1.45142734, ...,  0.27810695,
         0.12012696,  0.27810695],
       [ 1.4707842 ,  1.46472704,  1.45828363, ...,  0.39999634,
         0.27810695,  0.12012696]])

In [148]:
a = np.array([1,2,3,4,5])
c = 0.3*a
b = np.array([[1], [5], [4], [3], [1], [2]])
a == b

array([[ True, False, False, False, False],
       [False, False, False, False,  True],
       [False, False, False,  True, False],
       [False, False,  True, False, False],
       [ True, False, False, False, False],
       [False,  True, False, False, False]], dtype=bool)

In [143]:
np.argwhere(a == b)

array([[0, 0],
       [1, 4]], dtype=int32)

In [149]:
c[np.argwhere(a == b)[:,1]]

array([ 0.3,  1.5,  1.2,  0.9,  0.3,  0.6])

In [152]:
a = np.array([1,2,3,4,5])
c = 0.3*a
b = np.array([[1,5], [4,3], [1,2]])
a[...,np.newaxis] == b

C:\data_sci\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  after removing the cwd from sys.path.


False

In [151]:
a[...,np.newaxis]

array([[1],
       [2],
       [3],
       [4],
       [5]])

In [101]:
lims0

array([[ 0.025,  0.075,  0.125, ...,  1.875,  1.925,  1.975],
       [ 0.075,  0.025,  0.075, ...,  1.825,  1.875,  1.925],
       [ 0.125,  0.075,  0.025, ...,  1.775,  1.825,  1.875],
       ..., 
       [ 1.875,  1.825,  1.775, ...,  0.025,  0.075,  0.125],
       [ 1.925,  1.875,  1.825, ...,  0.075,  0.025,  0.075],
       [ 1.975,  1.925,  1.875, ...,  0.125,  0.075,  0.025]])

In [104]:
ulims[np.newaxis,...]

array([[ 0.025,  0.075,  0.125,  0.175,  0.225,  0.275,  0.325,  0.375,
         0.425,  0.475,  0.525,  0.575,  0.625,  0.675,  0.725,  0.775,
         0.825,  0.875,  0.925,  0.975,  1.025,  1.075,  1.125,  1.175,
         1.225,  1.275,  1.325,  1.375,  1.425,  1.475,  1.525,  1.575,
         1.625,  1.675,  1.725,  1.775,  1.825,  1.875,  1.925,  1.975]])

In [29]:
s = 7
slow_green = helper.get_green_matrix(well, s)
fast_green = helper.get_green_matrix_fast(well, s)
limits = helper.get_limits(well)

In [16]:
from time import time

In [26]:
ss = 1000*np.random.sample(1000)
ss = ss[ss != 0.]
t1 = time()
for s in ss:
    slow_green = helper.get_green_matrix(well, s)
slow_time = time() - t1
t1 = time()
for s in ss:
    fast_green = helper.get_green_matrix_fast(well ,s)
fast_time = time()-t1
print(slow_time, fast_time)

44.02051782608032 40.57732081413269


In [74]:
from scipy.special import kn, k0, k1, iti0k0

In [33]:
ss = 1000*np.random.sample(10000)
ss = ss[ss != 0.]
t1 = time()
for s in ss:
    slimits = s**0.5*limits
    x = iti0k0(slimits)
print(time()-t1)

0.4000229835510254


In [35]:
ss = 1000*np.random.sample(10000)
ss = ss[ss != 0.]
t1 = time()
for s in ss:
    ls = get_limits(well, s)
print(time()-t1)

54.9381422996521


In [42]:
ss = 1000*np.random.sample(10000)
ss = ss[ss != 0.]
t1 = time()
for s in ss:
    ls = get_limits(well, s)
    x = iti0k0(ls)
print(time()-t1)

80.23558926582336


In [37]:
a = np.arange(100).reshape(10,10)

In [40]:
a.r

AttributeError: 'numpy.ndarray' object has no attribute 'index'

In [80]:
s = 7
def try_matrix(well, s):
    N = well.params["nseg"]
    xwd = well.xwd
    dx = 1./N
    c = 1
    su = s**0.5
    m_xd = xwd + dx*np.matrix(np.arange(-N+0.5, N+0.5), dtype=np.float).transpose()*np.matrix(np.ones(2*N, dtype=np.float))
    m_xj1 = xwd + dx*np.matrix(np.ones(2*N, dtype=np.float)).transpose()*np.matrix(np.arange(-N, N), dtype=np.float)
    m_xj2 = dx + m_xj1
    lims1 = m_xd - c*m_xj1 - xwd
    lims2 = m_xd - c*m_xj2 - xwd
    mask_int_1 = np.matrix(np.ones((2*N, 2*N)), dtype=np.float) - 2*(lims1 < 0)
    mask_int_2 = np.matrix(np.ones((2*N, 2*N)), dtype=np.float) - 2*(lims2 >= 0)
    alims1 = np.abs(lims1)
    alims2 = np.abs(lims2)
    i1 = iti0k0(alims1)[1]
    i2 = iti0k0(alims2)[1]
    return np.multiply(i1, mask_int_1) + np.multiply(i2, mask_int_2)

In [81]:
ss = 1000*np.random.sample(10000)
ss = ss[ss != 0.]
t1 = time()
for s in ss:
    ints = try_matrix(well, s)
print(time()-t1)

19.245100736618042


In [70]:
m = np.matrix(np.arange(1,10).reshape(3,3))
b = np.matrix(np.ones((3,3)))
b[0,0], b[2,1] = -1 ,-1

In [209]:
a = np.array([1,2,3])

In [213]:
a.T@a

14

In [73]:
np.multiply(m,b)

matrix([[-1.,  2.,  3.],
        [ 4.,  5.,  6.],
        [ 7., -8.,  9.]])

In [65]:
np.matrix.prod?

In [208]:
type(well) == LaplWell

True